In [27]:
import sys 
import os
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.features import preprocessing
from pathlib import Path
from src.features import sequences
from src import ExperimentRunner

In [2]:
data_path = Path('/home/ralmasri/projects/Thesis/Domain-Guided-Monitoring/data/')
csv_path = data_path / "logs_aggregated_concurrent.csv"
size = 50
# data_df = pd.read_csv(csv_path).head(size) if size > 0  else pd.read_csv(csv_path)
subset_path = data_path / f"{size}_logs_aggregated_concurrent.csv" if size > 0 else csv_path
# if size > 0:
#     data_df.to_csv(subset_path)

In [3]:
huawei_config = preprocessing.HuaweiPreprocessorConfig()
huawei_config.aggregated_log_file = subset_path
sequence_preprocessor = preprocessing.ConcurrentAggregatedLogsPreprocessor(huawei_config)
sequence_column_name = sequence_preprocessor.sequence_column_name
sequence_df = sequence_preprocessor.load_data()
sequence_df

Generating DRAIN clusters from log_df: 100%|██████████| 15/15 [00:00<00:00, 1701.54it/s]


,num_logs,num_events,all_events,attributes,fine_log_cluster_template,coarse_log_cluster_template
0,50,50,"[[http_method#get, Hostname#wally113, http_sta...","[[http_method#get, Hostname#wally113, http_sta...",[[fine_log_cluster_template#19 nov 2019 17 * *...,[[coarse_log_cluster_template#19 nov 2019 17 *...


In [4]:
sequence_config = sequences.SequenceConfig()
sequence_config

SequenceConfig(test_percentage=0.1, random_test_split=True, random_state=12345, flatten_x=True, flatten_y=True, max_window_size=10, min_window_size=2, window_overlap=True, allow_subwindows=False, valid_y_features=[], remove_empty_y_vecs=True, remove_empty_x_vecs=True, x_sequence_column_name='', y_sequence_column_name='', predict_full_y_sequence=False, predict_full_y_sequence_wide=False)

In [9]:
sequence_preprocessor.sequence_column_name

'all_events'

In [20]:
transformer = sequences.NextSequenceTransformer(sequence_config)
metadata = transformer.collect_metadata(sequence_df=sequence_df, sequence_column_name='all_events')
metadata

In [34]:
runner = ExperimentRunner(1)
runner.sequence_df_pkl_file = str(data_path / "sequences_df.pkl")
runner.sequence_column_name = 'all_events'
(train_dataset, test_dataset) = runner._create_dataset(sequence_df)
train_dataset
X_train = list(map(lambda x: x[0], train_dataset))
y_train = list(map(lambda x: x[1], train_dataset))